# Loading Data ..

In [ ]:
pip install pydicom

In [ ]:
pip install wandb

In [ ]:
pip install kaggle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd '/content/drive/MyDrive/archive-3Original/COVID'

In [ ]:
ls

In [ ]:
pip install -U tensorflow-addons

In [ ]:
pwd

In [ ]:
import os
os.chdir('/content/drive/MyDrive/DatasetA/CP')
import os, fnmatch
def find(pattern, path):
    """Utility to find files wrt a regex search"""
    result = []
    for r, dirs, files in os.walk(path):
        for name in files:
            if fnmatch.fnmatch(name, pattern):
                result.append(os.path.join(r, name))
                #print(r, name)
    return result
FIND_FOLDER = '/content/drive/MyDrive/DatasetA/CP/-COVID-'
png_files = find('*.*', FIND_FOLDER)
print(len(png_files),"Files Found.")

5238 Files Found.


In [ ]:
import random
num_subset = 5238
#random.seed(42) # 42 # 2021
subset_png_files =  random.choices(png_files, k=num_subset) # dcm_files[:3]
subset_png_files

In [ ]:
import matplotlib.pyplot as plt
import cv2

# Define & Load UNET Model

In [ ]:
pip install -U tensorflow-addons

In [ ]:
!pip install -q keras

In [ ]:
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras import backend as keras
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, LearningRateScheduler


def dice_coef(y_true, y_pred):
    y_true_f = keras.flatten(y_true)
    y_pred_f = keras.flatten(y_pred)
    intersection = keras.sum(y_true_f * y_pred_f)
    return (2. * intersection + 1) / (keras.sum(y_true_f) + keras.sum(y_pred_f) + 1)

def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

def unet(input_size=(256,256,1)):
    inputs = Input(input_size)
    
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)

    up6 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5), conv4], axis=3)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(up6)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)

    up7 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6), conv3], axis=3)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(up7)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)

    up8 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7), conv2], axis=3)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(up8)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)

    up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8), conv1], axis=3)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(up9)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv9)

    conv10 = Conv2D(1, (1, 1), activation='sigmoid')(conv9)

    return Model(inputs=[inputs], outputs=[conv10])

In [ ]:

import tensorflow as tf
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
model = unet(input_size=(512,512,1))
model.compile(opt, loss=dice_coef_loss,
                  metrics=[dice_coef, 'binary_accuracy'])
model.summary()

**Load the Pretrained UNet Model**

In [ ]:
model_weights_path = "/content/drive/MyDrive/cxr_reg_png_weights.best.hdf5"

model.load_weights(model_weights_path)

In [ ]:
viz_counter=0


In [ ]:
"""
Shapes that you wish to resize to
"""
import numpy as np

image_dataset = []
Shape_X,Shape_Y=512,512

for i in subset_png_files:
    resized_image_data = cv2.resize(cv2.imread(os.path.join(FIND_FOLDER,i)),(Shape_Y,Shape_X))[:,:,0]
    #clahe = cv2.createCLAHE(cliplimit=3.0, tileGridSize=(8,8))
    #clahe_img = clahe.apply(resized_image_data)
    #color_clahe = cv2.merge((clahe_img,a,b))
    #CLAHE_img = cv2.cvtColor(color_clahe,cv2.COLOR_LAB2BGR)
    image_dataset.append(np.array(resized_image_data))
  
    ##dicom_dict, modified_image_data = dicom_dataset_to_dict(path)
    #resized_image_data = cv2.resize(modified_image_data,(Shape_Y,Shape_X)) # cv2 has this opposite
    # props(resized_image_data)
    prep_unet_input_img_1 = resized_image_data.reshape(1,Shape_X,Shape_Y,1)
    prep_unet_input_img = (prep_unet_input_img_1-127.0)/127.0
    pred_img = model.predict(prep_unet_input_img)
    pred_img_preprocessed_1 = np.squeeze(pred_img)
    pred_img_preprocessed = (pred_img_preprocessed_1*255>127).astype(np.int8)
    # props(pred_img_preprocessed)
    # print("Unique Values :",np.unique(pred_img_preprocessed))
    res = cv2.bitwise_and(resized_image_data,resized_image_data,mask = pred_img_preprocessed)
    fig, ax = plt.subplots(1, 3, figsize=(20, 12))
    ax[0].imshow(clahe_img, cmap="viridis")
    ax[0].axis('off')
    ax[1].imshow(pred_img_preprocessed, cmap="viridis")    
    ax[1].axis('off')
    ax[2].imshow(res, cmap="viridis")    
    ax[2].axis('off')
    #plt.savefig(str(viz_counter)+".png",dpi=300)
    viz_counter+=1
    cv2.imwrite(str(viz_counter)+".png",res)
    #viz_counter+=1
    plt.show()

In [ ]:
df = pd.DataFrame.from_dict({'image_id': image_id, 'dim0': dim0, 'dim1': dim1, 'split': splits})
df.to_csv('meta.csv', index=False)

In [ ]:
import os
import zipfile
import shutil

#taken from : https://www.kaggle.com/xhlulu/recursion-2019-load-resize-and-save-images

def zip_and_remove(path):
    ziph = zipfile.ZipFile(f'{path}.zip', 'w', zipfile.ZIP_DEFLATED)
    
    for root, dirs, files in os.walk(path):
        for file in files:
            file_path = os.path.join(root, file)
            ziph.write(file_path)
            os.remove(file_path)
    
    ziph.close()
    shutil.rmtree(path)
save_dir = '/content/drive/MyDrive/Segmented'
zip_and_remove(save_dir)

# Augmentation

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from skimage import io
import numpy as np
from PIL import Image

# Construct an instance of the ImageDataGenerator class
# Pass the augmentation parameters through the constructor. 

datagen = ImageDataGenerator(
        rotation_range=45,
        shear_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')    #Also try nearest, constant, reflect, wrap

dataset = []

import numpy as np
from skimage import io
import os
from PIL import Image

i = 0
for batch in datagen.flow_from_directory(directory='/content/drive/MyDrive/SegmentedDataset1/SegmentedUpdated/cap', 
                                         batch_size=1,  
                                         target_size=(256, 256),
                                         color_mode="rgb",
                                         save_to_dir='/content/drive/MyDrive/SegmentedDataset1/SegmentedUpdated/cap/CAP', 
                                         save_prefix='aug', 
                                         save_format='png'):
    i += 1
    if i > 2000:
        break 

